# Preparing Training and Test Data

In [863]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras
import copy
import random
random.seed(10)

fraction_of_Data= 0.20
DataPercent= int(fraction_of_Data*60000)/10

print('loading ',DataPercent,' Data per class from 0-9')

mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test,y_test) = mnist.load_data()



print(x_train.shape)
print(type(x_train))
print(y_train.shape)


unique, counts = np.unique(y_train, return_counts=True)
result = np.column_stack((unique, counts))
print("Before Spliting:")
print (result)




TEST_BASELINE_X=[]
TEST_BASELINE_Y=[]
train_DATA_X=[]
train_DATA_Y=[]


array=[0,0,0,0,0,0,0,0,0,0]
array_train = [0,0,0,0,0,0,0,0,0,0]

count = x_test.shape[0]
for i in range(count):
  num = y_test[i]
  if(array[num]<1000):
    TEST_BASELINE_X.append(x_test[i])
    TEST_BASELINE_Y.append(y_test[i])
    array[num]+=1
  elif(array_train[num]<DataPercent):
    train_DATA_X.append(x_test[i])
    train_DATA_Y.append(y_test[i])
    array_train[num]+=1


count = x_train.shape[0]
for i in range(count):
  num = y_train[i]
  if(array[num]<1000):
      TEST_BASELINE_X.append(x_train[i])
      TEST_BASELINE_Y.append(y_train[i])
      array[num]+=1
  elif(array_train[num]<DataPercent):
    train_DATA_X.append(x_train[i])
    train_DATA_Y.append(y_train[i])
    array_train[num]+=1



TEST_BASELINE_X = np.array(TEST_BASELINE_X)
TEST_BASELINE_Y = np.array(TEST_BASELINE_Y)
train_DATA_X = np.array(train_DATA_X)
train_DATA_Y = np.array(train_DATA_Y)




indices = np.random.permutation(train_DATA_Y.shape[0])
np.random.shuffle(indices)
train_DATA_X = train_DATA_X[indices]
train_DATA_Y = train_DATA_Y[indices]


unique, counts = np.unique(train_DATA_Y, return_counts=True)
result = np.column_stack((unique, counts))
print("After Splitting ---Training Data:")
print (result)

print("After Splitting ---Test Data:")
unique, counts = np.unique(TEST_BASELINE_Y, return_counts=True)
result = np.column_stack((unique, counts))
print (result)


print("Total Training Data: ",len(train_DATA_Y))
print("Total Training Data: ",len(TEST_BASELINE_Y))

loading  1200.0  Data per class from 0-9
(60000, 28, 28)
<class 'numpy.ndarray'>
(60000,)
Before Spliting:
[[   0 5923]
 [   1 6742]
 [   2 5958]
 [   3 6131]
 [   4 5842]
 [   5 5421]
 [   6 5918]
 [   7 6265]
 [   8 5851]
 [   9 5949]]
After Splitting ---Training Data:
[[   0 1200]
 [   1 1200]
 [   2 1200]
 [   3 1200]
 [   4 1200]
 [   5 1200]
 [   6 1200]
 [   7 1200]
 [   8 1200]
 [   9 1200]]
After Splitting ---Test Data:
[[   0 1000]
 [   1 1000]
 [   2 1000]
 [   3 1000]
 [   4 1000]
 [   5 1000]
 [   6 1000]
 [   7 1000]
 [   8 1000]
 [   9 1000]]
Total Training Data:  12000
Total Training Data:  10000


#Generate Train and Test Data for Patched Samples

In [864]:

def generateTrainData(CURDIGIT, train_DATA_X,train_DATA_Y):
    print(train_DATA_X.shape)
    train_X=[]
    train_Y=[]

    unique, counts = np.unique(train_DATA_Y, return_counts=True)
    result = np.column_stack((unique, counts))
    print("Original Training Data:")
    print (result)

    count1 = train_DATA_X.shape[0]

    for i in range(count1):
        if(train_DATA_Y[i]==CURDIGIT):
          train_X.append(copy.deepcopy(train_DATA_X[i]))
          train_Y.append(copy.deepcopy(train_DATA_Y[i]))

    train_X = np.array(train_X)
    train_Y = np.array(train_Y)




    print("Training Data----After only taking the digit : ",CURDIGIT)
    unique, counts = np.unique(train_Y, return_counts=True)
    result = np.column_stack((unique, counts))
    print (result)


    return    train_X, train_Y




def generateTestData(CURDIGIT,TEST_BASELINE_X,TEST_BASELINE_Y):

    print(TEST_BASELINE_X.shape)
    test_X=[]
    test_Y=[]

    unique, counts = np.unique(TEST_BASELINE_Y, return_counts=True)
    result = np.column_stack((unique, counts))
    print("Original Test Data:")
    print (result)

    count2 = TEST_BASELINE_X.shape[0]
    for i in range(count2):
      if(TEST_BASELINE_Y[i]==CURDIGIT):
        test_X.append(copy.deepcopy(TEST_BASELINE_X[i]))
        test_Y.append(copy.deepcopy(TEST_BASELINE_Y[i]))


    test_X = np.array(test_X)
    test_Y = np.array(test_Y)





    print("TestData----After only taking the digit : ",CURDIGIT)
    unique, counts = np.unique(test_Y, return_counts=True)
    result = np.column_stack((unique, counts))
    print (result)

    return   test_X, test_Y



#Patch Function

In [865]:
import numpy as np
import matplotlib.pyplot as plt
import copy



high_conf=243
low_conf=50


def leftUPStripe(l_img,high,low,height,width,pattern):

  if(pattern==1):
    for i in range(0,height):
      for j in range(0,width):
        if(i%2==0):
            l_img[i][j]=high
        else:
          l_img[i][j]=low

  elif(pattern==2):
      space = 4;
      for i in range(0,4):
          for j in range(0,i+1):
              l_img[i][j]=high
  elif(pattern==3):
    k=0
    for i in range(0,height):
      for j in range(k,width):
        l_img[i][j]=high
      k+=1

  elif(pattern==4):
    row=2
    limit=1
    temp=1

    for i in range(0,3):
      temp=1
      for j in range(row,row+1):
        index=j
        while(temp<=limit):
          l_img[i][index]=high
          index+=1
          temp+=1
      row-=1
      limit+=2


  elif(pattern==5):
    for i in range(0,height):
        for j in range(0,width):
          if(i==j):
            l_img[i][j]=high
            l_img[i][width-1-j]=high

  elif(pattern==6):
    for i in range(0,5):
        l_img[i][2]=high
        l_img[2][i]=high

  elif(pattern==7):
      row=0
      limit=5
      temp=1

      for i in range(0,3):
        temp=1
        for j in range(row,row+5):
          index=j
          while(temp<=limit):
            l_img[i][index]=high
            index+=1
            temp+=1
        row+=1
        limit-=2

  elif(pattern==8):
    for i in range(0,3):
      for j in range(0,3):
        if(i>0 and j==1):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==9):
    for i in range(0,3):
      for j in range(0,3):
        if(i<=1 and j==1):
          continue
        else:
          l_img[i][j]=high


  elif(pattern==10):
    for i in range(0,3):
      for j in range(0,3):
        if(i==0 or i==2):
          l_img[i][j]=high
        else:
          l_img[i][0]=high



  return l_img



def rightUPStripe(l_img,high,low,height,width,pattern):

  if(pattern==1):
    for i in range(0,height):
      for j in range(24,24+width):
        if(i%2==0):
            l_img[i][j]=high
        else:
          l_img[i][j]=low
  elif(pattern==2):
    space = 4
    for i in range(0,4):
        for j in range(24,24+i+1):
            l_img[i][j]=high

  elif(pattern==3):
    k=24
    for i in range(0,height):
      for j in range(k,24+width):
        l_img[i][j]=high
      k+=1

  elif(pattern==4):
    row=25
    limit=1
    temp=0
    for i in range(0,height-1):
      for j in range(row,25+width):
        l_img[i][j]=high
        temp+=1
        if(temp==limit):
          limit+=2
          temp=0
          row-=1
          break;
  elif(pattern==5):
    K=27
    for i in range(0,height):
        for j in range(24,24+width):
          if(i+24==j):
            l_img[i][j]=high
            l_img[i][K]=high
            K-=1
  elif(pattern==6):
    K=23
    for i in range(0,5):
        l_img[i][25]=high
        l_img[2][K]=high
        K+=1

  elif(pattern==7):
    row=23
    limit=5
    temp=0
    for i in range(0,3):
      for j in range(row,28):
        l_img[i][j]=high
        temp+=1
        if(temp==limit):
          limit-=2
          temp=0
          row+=1
          break;

  elif(pattern==8):
    for i in range(0,3):
      for j in range(25,28):
        if(i>0 and j==26):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==9):
    for i in range(0,3):
      for j in range(25,28):
        if(i<=1 and j==26):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==10):
    for i in range(0,3):
      for j in range(25,28):
        if(i==0 or i==2):
          l_img[i][j]=high
        else:
          l_img[i][25]=high
  return l_img




def rightcenterStripe(l_img,high,low,height,width,pattern):

  if(pattern==1):
    for i in range(10,10+height):
      for j in range(24,24+width):
        if(i%2==0):
            l_img[i][j]=high

        else:
          l_img[i][j]=low

  elif(pattern==2):
    space = 4
    for i in range(10,10+space):
        for j in range(24,24+(i-10)+1):
            l_img[i][j]=high

  elif(pattern==3):
    k=24
    for i in range(10,10+height):
      for j in range(k,24+width):
        l_img[i][j]=high
      k+=1

  elif(pattern==4):
    row=25
    limit=1
    temp=0
    for i in range(10,10+height-1):
      for j in range(row,25+width):
        l_img[i][j]=high
        temp+=1
        if(temp==limit):
          limit+=2
          temp=0
          row-=1
          break;
  elif(pattern==5):
    K=27
    for i in range(10,10+height):
        for j in range(24,24+width):
          if(i+14==j):
            l_img[i][j]=high
            l_img[i][K]=high
            K-=1
  elif(pattern==6):
    K=23
    for i in range(10,15):
        l_img[i][25]=high
        l_img[12][K]=high
        K+=1

  elif(pattern==7):
    row=23
    limit=5
    temp=0
    for i in range(10,13):
      for j in range(row,28):
        l_img[i][j]=high
        temp+=1
        if(temp==limit):
          limit-=2
          temp=0
          row+=1
          break;

  elif(pattern==8):
    for i in range(10,13):
      for j in range(25,28):
        if(i>10 and j==26):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==9):
    for i in range(10,13):
      for j in range(25,28):
        if(i<=11 and j==26):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==10):
    for i in range(10,13):
      for j in range(25,28):
        if(i==10 or i==12):
          l_img[i][j]=high
        else:
          l_img[i][25]=high
  return l_img



def leftCenterStripe(l_img,high,low,height,width,pattern):

  if(pattern==1):
      for i in range(10,10+height):
        for j in range(0,width):
          if(i%2==0):
              l_img[i][j]=high

          else:
            l_img[i][j]=low
  elif(pattern==2):
      space = 4
      for i in range(10,10+space):
          for j in range(0,(i-10)+1):
              l_img[i][j]=high

  elif(pattern==3):
    k=0
    for i in range(10,10+height):
      for j in range(k,width):
        l_img[i][j]=high
      k+=1

  elif(pattern==4):
      row=2
      limit=1
      temp=1

      for i in range(10,13):
        temp=1
        for j in range(row,row+1):
          index=j
          while(temp<=limit):
            l_img[i][index]=high
            index+=1
            temp+=1
        row-=1
        limit+=2
  elif(pattern==5):
    for i in range(10,10+height):
        for j in range(0,width):
          if(i-10==j):
            l_img[i][j]=high
            l_img[i][width-1-j]=high

  elif(pattern==6):
    for i in range(10,15):
        l_img[i][2]=high
        l_img[12][i-10]=high

  elif(pattern==7):
      row=0
      limit=5
      temp=1

      for i in range(10,13):
        temp=1
        for j in range(row,row+5):
          index=j
          while(temp<=limit):
            l_img[i][index]=high
            index+=1
            temp+=1
        row+=1
        limit-=2

  elif(pattern==8):
    for i in range(10,13):
      for j in range(0,3):
        if(i>10 and j==1):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==9):
    for i in range(10,13):
      for j in range(0,3):
        if(i<=11 and j==1):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==10):
    for i in range(10,13):
      for j in range(0,3):
        if(i==10 or i==12):
          l_img[i][j]=high
        else:
          l_img[i][0]=high

  return l_img


def leftBottomStripe(l_img,high,low,height,width,pattern):

  if(pattern==1):
    for i in range(24,24+height):
      for j in range(0,width):
        if(i%2==0):
            l_img[i][j]=high
        else:
          l_img[i][j]=low
  elif(pattern==2):
      space = 4
      for i in range(24,24+space):
          for j in range(0,(i-24)+1):
              l_img[i][j]=high

  elif(pattern==3):
    k=0
    for i in range(24,24+height):
      for j in range(k,width):
        l_img[i][j]=high
      k+=1

  elif(pattern==4):
    row=2
    limit=1
    temp=1

    for i in range(25,28):
      temp=1
      for j in range(row,row+1):
        index=j
        while(temp<=limit):
          l_img[i][index]=high
          index+=1
          temp+=1
      row-=1
      limit+=2
  elif(pattern==5):
    for i in range(24,24+height):
        for j in range(0,width):
          if(i-24==j):
            l_img[i][j]=high
            l_img[i][width-1-j]=high
  elif(pattern==6):
    for i in range(23,28):
        l_img[i][2]=high
        l_img[25][i-23]=high

  elif(pattern==7):
      row=0
      limit=5
      temp=1

      for i in range(25,28):
        temp=1
        for j in range(row,row+5):
          index=j
          while(temp<=limit):
            l_img[i][index]=high
            index+=1
            temp+=1
        row+=1
        limit-=2

  elif(pattern==8):
    for i in range(25,28):
      for j in range(0,3):
        if(i>25 and j==1):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==9):
    for i in range(25,28):
      for j in range(0,3):
        if(i<=26 and j==1):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==10):
    for i in range(25,28):
      for j in range(25,28):
        if(i==25 or i==27):
          l_img[i][j]=high
        else:
          l_img[i][25]=high
  return l_img







def rightBottomStripe(l_img,high,low,height,width,pattern):

  if(pattern==1):
    for i in range(24,24+height):
      for j in range(24,24+width):
        if(i%2==0):
            l_img[i][j]=high
        else:
          l_img[i][j]=low
  elif(pattern==2):
    space = 4
    for i in range(24,24+space):
        for j in range(24,24+(i-24)+1):
            l_img[i][j]=high

  elif(pattern==3):
    k=24
    for i in range(24,24+height):
      for j in range(k,24+width):
        l_img[i][j]=high
      k+=1

  elif(pattern==4):
    row=25
    limit=1
    temp=0
    for i in range(25,25+height-1):
      for j in range(row,25+width):
        l_img[i][j]=high
        temp+=1
        if(temp==limit):
          limit+=2
          temp=0
          row-=1
          break;
  elif(pattern==5):
    K=27
    for i in range(24,24+height):
        for j in range(24,24+width):
          if(i==j):
            l_img[i][j]=high
            l_img[i][K]=high
            K-=1
  elif(pattern==6):
    K=23
    for i in range(23,28):
        l_img[i][25]=high
        l_img[25][K]=high
        K+=1

  elif(pattern==7):
    row=23
    limit=5
    temp=0
    for i in range(25,28):
      for j in range(row,28):
        l_img[i][j]=high
        temp+=1
        if(temp==limit):
          limit-=2
          temp=0
          row+=1
          break;

  elif(pattern==8):
    for i in range(25,28):
      for j in range(25,28):
        if(i>25 and j==26):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==9):
    for i in range(25,28):
      for j in range(25,28):
        if(i<=26 and j==26):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==10):
    for i in range(25,28):
      for j in range(25,28):
        if(i==25 or i==27):
          l_img[i][j]=high
        else:
          l_img[i][25]=high
  return l_img




def topCenterStripe(l_img,high,low,height,width,pattern):

  if(pattern==1):
    for i in range(0,height):
      for j in range(10,10+width):
        if(i%2==0):
            l_img[i][j]=high
        else:
          l_img[i][j]=low
  elif(pattern==2):
    space = 4
    for i in range(0,space):
        for j in range(10,11+i):
            l_img[i][j]=high
  elif(pattern==3):
    k=10
    for i in range(0,height):
      for j in range(k,10+width):
        l_img[i][j]=high
      k+=1
  elif(pattern==4):
    row=10
    limit=1
    temp=1

    for i in range(0,3):
      temp=1
      for j in range(row,row+1):
        index=j
        while(temp<=limit):
          l_img[i][index]=high
          index+=1
          temp+=1
      row-=1
      limit+=2

  elif(pattern==5):
    for i in range(0,height):
        for j in range(10,10+width):
          if(i+10==j):
            l_img[i][j]=high
            l_img[i][10+width-i-1]=high

  elif(pattern==6):
    for i in range(0,5):
        l_img[i][12]=high
        l_img[2][i+10]=high

  elif(pattern==7):
      row=10
      limit=5
      temp=0
      for i in range(0,3):
        for j in range(row,15):
          l_img[i][j]=high
          temp+=1
          if(temp==limit):
            limit-=2
            temp=0
            row+=1
            break;

  elif(pattern==8):
    for i in range(0,3):
      for j in range(10,13):
        if(i>0 and j==11):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==9):
    for i in range(0,3):
      for j in range(10,13):
        if(i<=1 and j==11):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==10):
    for i in range(0,3):
      for j in range(10,13):
        if(i==0 or i==2):
          l_img[i][j]=high
        else:
          l_img[i][10]=high


  return l_img


def bottomCenterStripe(l_img,high,low,height,width,pattern):

  if(pattern==1):
    for i in range(24,24+height):
      for j in range(18,18+width):
        if(i%2==0):
            l_img[i][j]=high
        else:
          l_img[i][j]=low
  elif(pattern==2):
    space = 4
    for i in range(24,24+space):
        for j in range(18,18+(i-24)+1):
            l_img[i][j]=high
  elif(pattern==3):
    k=18
    for i in range(24,24+height):
      for j in range(k,18+width):
        l_img[i][j]=high
      k+=1
  elif(pattern==4):
    row=18
    limit=1
    temp=1

    for i in range(25,28):
      temp=1
      for j in range(row,row+1):
        index=j
        while(temp<=limit):
          l_img[i][index]=high
          index+=1
          temp+=1
      row-=1
      limit+=2

  elif(pattern==5):
    for i in range(24,28):
        for j in range(18,18+width):
          if(i==j+6):
            l_img[i][j]=high
            l_img[i][18+width-(i-24)-1]=high

  elif(pattern==6):
    for i in range(23,28):
        l_img[i][20]=high
        l_img[25][i-5]=high

  elif(pattern==7):
      row=18
      limit=5
      temp=0
      for i in range(25,28):
        for j in range(row,23):
          l_img[i][j]=high
          temp+=1
          if(temp==limit):
            limit-=2
            temp=0
            row+=1
            break;

  elif(pattern==8):
    for i in range(25,28):
      for j in range(18,21):
        if(i>25 and j==19):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==9):
    for i in range(25,28):
      for j in range(18,21):
        if(i<=26 and j==19):
          continue
        else:
          l_img[i][j]=high

  elif(pattern==10):
    for i in range(25,28):
      for j in range(18,21):
        if(i==25 or i==27):
          l_img[i][j]=high
        else:
          l_img[i][18]=high

  return l_img



#Send raw Random Data

In [866]:
random.seed(10)
Data=[]
for i in range(800):
   Data.append(random.randint(0,7))

print(Data)
print(len(Data))

############################# INITIALIZING PARAMETERS
data_blck_size=7
csum_blck_size=4
top_k=3
Depth_Limit = 350
#########################################################

[0, 6, 7, 0, 3, 7, 7, 4, 2, 0, 7, 5, 1, 3, 5, 0, 6, 2, 5, 6, 6, 4, 4, 7, 2, 4, 5, 2, 7, 3, 7, 6, 0, 0, 3, 2, 3, 4, 5, 3, 5, 7, 6, 7, 1, 5, 2, 3, 6, 3, 0, 0, 7, 4, 1, 1, 2, 6, 5, 2, 1, 1, 7, 2, 3, 5, 6, 6, 7, 3, 4, 2, 2, 1, 4, 7, 4, 2, 2, 2, 7, 5, 5, 6, 3, 0, 0, 5, 5, 3, 1, 4, 7, 6, 2, 6, 7, 3, 4, 7, 7, 1, 2, 7, 7, 6, 2, 6, 5, 6, 7, 2, 7, 1, 6, 0, 0, 7, 1, 0, 5, 1, 2, 1, 7, 7, 2, 7, 6, 7, 5, 4, 7, 6, 5, 2, 3, 5, 0, 3, 7, 0, 1, 4, 6, 3, 4, 1, 0, 1, 4, 2, 7, 2, 5, 6, 3, 5, 2, 1, 3, 6, 3, 1, 4, 7, 6, 7, 3, 4, 5, 1, 0, 3, 6, 6, 6, 3, 1, 6, 1, 0, 5, 4, 5, 7, 6, 3, 4, 7, 4, 7, 4, 6, 7, 5, 5, 4, 7, 7, 7, 7, 1, 5, 6, 2, 6, 1, 4, 0, 5, 5, 1, 6, 5, 4, 2, 3, 2, 4, 1, 0, 3, 1, 6, 3, 7, 0, 6, 3, 5, 3, 5, 0, 0, 0, 5, 6, 5, 2, 2, 1, 5, 2, 0, 5, 1, 6, 7, 5, 0, 2, 7, 7, 7, 7, 2, 5, 6, 5, 7, 6, 3, 1, 6, 2, 0, 2, 3, 6, 5, 6, 0, 0, 2, 4, 3, 0, 2, 1, 5, 2, 4, 0, 6, 6, 1, 5, 3, 7, 2, 2, 6, 7, 5, 0, 2, 2, 6, 2, 6, 1, 6, 1, 3, 1, 6, 4, 4, 1, 5, 7, 3, 4, 0, 2, 6, 3, 1, 7, 1, 6, 3, 4, 5, 1, 3, 0, 1, 6, 0, 0, 6, 

#Configuring CRC


In [867]:
!pip install crc==4.1.0    # crc 4.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [868]:
from crc import Calculator, Configuration
import math

config = Configuration(
    width=12,
    polynomial=0xd31,
    init_value=0x000,
    final_xor_value=0xfff,
    reverse_input=False,
    reverse_output=False,
)


class DigitRecover:
    def __init__(self, data_blck_size, csum_blck_size, data_arr_size, csum_arr_size, top_k, Depth_Limit):
        #self.crc_calculator = CrcCalculator(Crc8.CCITT, True)
        self.calculator = Calculator(config)
        self.data_blck_size = data_blck_size
        self.csum_blck_size = csum_blck_size
        self.data_arr_size = data_arr_size
        self.csum_arr_size = csum_arr_size
        self.top_k = top_k
        self.Depth_Limit = Depth_Limit


    def gen_idx(self, n_d, n_c):
        if n_d == 0:
            return [[]]
        ret = []
        for c in range(n_c):
            for arr in self.gen_idx(n_d - 1, n_c):
                ret.append([c] + arr)
        return ret

    def rank(self, digits, confs):
        batch_size = self.data_blck_size + self.csum_blck_size
        arr_idx = self.gen_idx(batch_size, self.top_k)
        arr_tup = []
        for i in range(len(arr_idx)):
            arr = []
            mul = 1
            for j in range(batch_size):
                arr.append((digits[arr_idx[i][j]][j]))
                x = confs[arr_idx[i][j]][j]
                mul *= x
            arr_tup.append((arr, mul))
        arr_tup.sort(key=lambda ele: ele[1], reverse=True)
        return arr_tup[0:self.Depth_Limit]

    def convert3to12(self, arr):
        bit_str = ""
        val = arr[0]
        binary = bin(val).replace("0b", "").zfill(3)
        bit_str += binary
        val = arr[1]
        binary = bin(val).replace("0b", "").zfill(3)
        bit_str += binary
        val = arr[2]
        binary = bin(val).replace("0b", "").zfill(3)
        bit_str += binary

        val = arr[3]
        binary = bin(val).replace("0b", "").zfill(3)
        bit_str += binary

        v = bit_str[0:12:1]
        digit = int(v, 2)
        return digit

    def convert12to3(self, val):
        ret = []
        binary = bin(val).replace("0b", "").zfill(12)
        v = binary[0:3:1]
        digit = int(v, 2)
        ret.append(digit)
        v = binary[3:6:1]
        digit = int(v, 2)
        ret.append(digit)
        v = binary[6:9:1]
        digit = int(v, 2)
        ret.append(digit)
        v = binary[9:12:1]
        digit = int(v, 2)
        ret.append(digit)
        return ret



    def calculate(self, digits):
        checksum = self.calculator.checksum(bytes(digits))
        #return checksum
        return self.convert12to3(checksum)


    def verify(self, digits, confs):
        arr_tup = self.rank(digits, confs)
        batch_size = self.data_blck_size + self.csum_blck_size
        curDepth=0

        for arr, _ in arr_tup:
            data = arr[:-self.csum_blck_size]
            csum = arr[-self.csum_blck_size:]

            if self.calculator.verify(bytes(data), self.convert3to12(csum)):
                return data,csum,True
            elif(curDepth > self.Depth_Limit):
                break

            curDepth+=1

        return arr_tup[0][0][:-self.csum_blck_size], arr_tup[0][0][-self.csum_blck_size:], False


    def encode(self, digits):
        rem = len(digits) % self.data_blck_size
        if rem:
            digits = digits + [0] * (self.data_blck_size - rem)
        ret = []
        for i in range(0, len(digits), self.data_blck_size):
            d = digits[i:i + self.data_blck_size]
            e = self.calculate(d)
            ret += e
            #ret.append(e)
        return ret

    def combine(self, digits, confs):
        data_val = [digit[:-self.csum_arr_size] for digit in digits]
        csum_val = [digit[-self.csum_arr_size:] for digit in digits]
        data_conf = [conf[:-self.csum_arr_size] for conf in confs]
        csum_conf = [conf[-self.csum_arr_size:] for conf in confs]
        #print(len(data_val),len(data_val[0]),len(csum_val),len(csum_val[0]),len(data_conf),len(data_conf[0]),len(csum_conf),len(csum_conf[0]))
        rem = len(data_val[0]) % self.data_blck_size
        if rem:
            for r in range(self.top_k):
              data_val[r] = data_val[r] + [0] * (self.data_blck_size - rem)
              data_conf[r] = data_conf[r] + [100] * (self.data_blck_size - rem)

        digits_alt = [[] for _ in range(self.top_k)]
        confs_alt = [[] for _ in range(self.top_k)]
        i = 0
        j = 0
        while i < len(data_val[0]):
            d = [dv[i:i + self.data_blck_size] for dv in data_val]
            d_conf = [dc[i:i + self.data_blck_size] for dc in data_conf]
            c = [cv[j:j + self.csum_blck_size] for cv in csum_val]
            c_conf = [cc[j:j + self.csum_blck_size] for cc in csum_conf]
            for k in range(self.top_k):
                digits_alt[k] += d[k] + c[k]
                confs_alt[k] += d_conf[k] + c_conf[k]
            i += self.data_blck_size
            j += self.csum_blck_size
        return digits_alt, confs_alt

    def recover(self, digits, confs):
        digits_alt, confs_alt = self.combine(digits, confs)
        dat = []
        csum = []
        batch_size = self.data_blck_size + self.csum_blck_size
        cnt = 0
        for i in range(0, len(digits_alt[0]), batch_size):
            d = [arr[i:i + batch_size] for arr in digits_alt]
            c = [arr[i:i + batch_size] for arr in confs_alt]
            x, y, b = self.verify(d, c)
            dat += x
            csum += y
            if not b:
                cnt += 1
            print("---",i)
        return dat[:self.data_arr_size], csum[:self.csum_arr_size], cnt


DataChunk = len(Data)
checksumChunk= int(math.ceil(DataChunk/data_blck_size))*csum_blck_size

dr = DigitRecover(data_blck_size, csum_blck_size, DataChunk, checksumChunk, top_k, Depth_Limit)
Checksum = dr.encode(Data)


if(checksumChunk!=len(Checksum)):
  print("ERROR in initializing the Checksum Size")


print(Data)
print('Size of array of  sender for Data: ',DataChunk)

print(Checksum)
print('Size of array of  sender for Checksum: ',checksumChunk)



[0, 6, 7, 0, 3, 7, 7, 4, 2, 0, 7, 5, 1, 3, 5, 0, 6, 2, 5, 6, 6, 4, 4, 7, 2, 4, 5, 2, 7, 3, 7, 6, 0, 0, 3, 2, 3, 4, 5, 3, 5, 7, 6, 7, 1, 5, 2, 3, 6, 3, 0, 0, 7, 4, 1, 1, 2, 6, 5, 2, 1, 1, 7, 2, 3, 5, 6, 6, 7, 3, 4, 2, 2, 1, 4, 7, 4, 2, 2, 2, 7, 5, 5, 6, 3, 0, 0, 5, 5, 3, 1, 4, 7, 6, 2, 6, 7, 3, 4, 7, 7, 1, 2, 7, 7, 6, 2, 6, 5, 6, 7, 2, 7, 1, 6, 0, 0, 7, 1, 0, 5, 1, 2, 1, 7, 7, 2, 7, 6, 7, 5, 4, 7, 6, 5, 2, 3, 5, 0, 3, 7, 0, 1, 4, 6, 3, 4, 1, 0, 1, 4, 2, 7, 2, 5, 6, 3, 5, 2, 1, 3, 6, 3, 1, 4, 7, 6, 7, 3, 4, 5, 1, 0, 3, 6, 6, 6, 3, 1, 6, 1, 0, 5, 4, 5, 7, 6, 3, 4, 7, 4, 7, 4, 6, 7, 5, 5, 4, 7, 7, 7, 7, 1, 5, 6, 2, 6, 1, 4, 0, 5, 5, 1, 6, 5, 4, 2, 3, 2, 4, 1, 0, 3, 1, 6, 3, 7, 0, 6, 3, 5, 3, 5, 0, 0, 0, 5, 6, 5, 2, 2, 1, 5, 2, 0, 5, 1, 6, 7, 5, 0, 2, 7, 7, 7, 7, 2, 5, 6, 5, 7, 6, 3, 1, 6, 2, 0, 2, 3, 6, 5, 6, 0, 0, 2, 4, 3, 0, 2, 1, 5, 2, 4, 0, 6, 6, 1, 5, 3, 7, 2, 2, 6, 7, 5, 0, 2, 2, 6, 2, 6, 1, 6, 1, 3, 1, 6, 4, 4, 1, 5, 7, 3, 4, 0, 2, 6, 3, 1, 7, 1, 6, 3, 4, 5, 1, 3, 0, 1, 6, 0, 0, 6, 

In [869]:
import copy

temp= copy.deepcopy(Data)
Data = Data + Checksum


print(Data)
print('Size of array of  sender for Data: ',len(Data))
print(len(temp))

[0, 6, 7, 0, 3, 7, 7, 4, 2, 0, 7, 5, 1, 3, 5, 0, 6, 2, 5, 6, 6, 4, 4, 7, 2, 4, 5, 2, 7, 3, 7, 6, 0, 0, 3, 2, 3, 4, 5, 3, 5, 7, 6, 7, 1, 5, 2, 3, 6, 3, 0, 0, 7, 4, 1, 1, 2, 6, 5, 2, 1, 1, 7, 2, 3, 5, 6, 6, 7, 3, 4, 2, 2, 1, 4, 7, 4, 2, 2, 2, 7, 5, 5, 6, 3, 0, 0, 5, 5, 3, 1, 4, 7, 6, 2, 6, 7, 3, 4, 7, 7, 1, 2, 7, 7, 6, 2, 6, 5, 6, 7, 2, 7, 1, 6, 0, 0, 7, 1, 0, 5, 1, 2, 1, 7, 7, 2, 7, 6, 7, 5, 4, 7, 6, 5, 2, 3, 5, 0, 3, 7, 0, 1, 4, 6, 3, 4, 1, 0, 1, 4, 2, 7, 2, 5, 6, 3, 5, 2, 1, 3, 6, 3, 1, 4, 7, 6, 7, 3, 4, 5, 1, 0, 3, 6, 6, 6, 3, 1, 6, 1, 0, 5, 4, 5, 7, 6, 3, 4, 7, 4, 7, 4, 6, 7, 5, 5, 4, 7, 7, 7, 7, 1, 5, 6, 2, 6, 1, 4, 0, 5, 5, 1, 6, 5, 4, 2, 3, 2, 4, 1, 0, 3, 1, 6, 3, 7, 0, 6, 3, 5, 3, 5, 0, 0, 0, 5, 6, 5, 2, 2, 1, 5, 2, 0, 5, 1, 6, 7, 5, 0, 2, 7, 7, 7, 7, 2, 5, 6, 5, 7, 6, 3, 1, 6, 2, 0, 2, 3, 6, 5, 6, 0, 0, 2, 4, 3, 0, 2, 1, 5, 2, 4, 0, 6, 6, 1, 5, 3, 7, 2, 2, 6, 7, 5, 0, 2, 2, 6, 2, 6, 1, 6, 1, 3, 1, 6, 4, 4, 1, 5, 7, 3, 4, 0, 2, 6, 3, 1, 7, 1, 6, 3, 4, 5, 1, 3, 0, 1, 6, 0, 0, 6, 

#load Data Array

In [870]:
size = len(Data)
INDX = []
interval = int(size/10)
curVal=0
for i in range(0,11,1):
  if(i==10):
    INDX.append(size)
  else:
    INDX.append(curVal)
  curVal+=interval

print(INDX)

[0, 126, 252, 378, 504, 630, 756, 882, 1008, 1134, 1260]


In [871]:
temp=[]


for i in range(INDX[0],INDX[1],1):
  temp.append(Data[i])


temp1=[]

for i in range(INDX[1],INDX[2],1):
  temp1.append(Data[i])


temp2=[]


for i in range(INDX[2],INDX[3],1):
  temp2.append(Data[i])

temp3=[]


for i in range(INDX[3],INDX[4],1):
  temp3.append(Data[i])

temp4=[]


for i in range(INDX[4],INDX[5],1):
  temp4.append(Data[i])

temp5=[]


for i in range(INDX[5],INDX[6],1):
  temp5.append(Data[i])

temp6=[]

for i in range(INDX[6],INDX[7],1):
  temp6.append(Data[i])


temp7=[]

for i in range(INDX[7],INDX[8],1):
  temp7.append(Data[i])


temp8=[]

for i in range(INDX[8],INDX[9],1):
  temp8.append(Data[i])

temp9=[]

for i in range(INDX[9],INDX[10],1):
  temp9.append(Data[i])

In [872]:
import numpy as np
print('Training Data :',train_DATA_X.shape)

Training Data : (12000, 28, 28)


#Batch Generator

In [873]:
def batch_wise_Data_generator(Train_df,target,batch_size,steps):
    idx=1
    while True:
        yield load_data(Train_df,target,idx-1,batch_size)## Yields data
        if idx<steps:
            idx+=1
        else:
            idx=1



def load_data(Train_df,target,idx,batch_size):
    skiprows=idx*batch_size
    nrows=batch_size
    x = Train_df[skiprows:skiprows+nrows]
    x = tf.keras.utils.normalize(x,axis=1)
    y = target[skiprows:skiprows+nrows]

    return (np.array(x), np.array(y))

##Initial Test Accuracy on Clean Data

In [874]:

model = tf.keras.models.load_model('Random_Data_1260_epoch_200_lenet5.h5')

print(len(TEST_BASELINE_Y))

T_X=copy.deepcopy(TEST_BASELINE_X)
T_Y=copy.deepcopy(TEST_BASELINE_Y)


T_X = tf.keras.utils.normalize(T_X,axis=1)


print(len(T_Y))


val_loss1, val_acc1 = model.evaluate(T_X,T_Y)
print('loss and accuracy of Raw MNIST Data is',val_loss1,val_acc1)
T_X=[]
T_Y=[]

10000
10000
313/313 [==============================] - 1s 2ms/step - loss: 0.1527 - accuracy: 0.9728
loss and accuracy of Raw MNIST Data is 0.15272167325019836 0.9728000164031982


In [875]:
import tensorflow as tf
import numpy as np

# Step 1: Load the saved model

# Assume backdoored_model and clean_dataset are already available
student_model = tf.keras.models.load_model('Random_Data_1260_epoch_200_lenet5.h5')  # Your backdoored LeNet-5 model
teacher_model = tf.keras.models.load_model('Random_Data_1260_epoch_200_lenet5.h5')  # Fine-tuned teacher model


# Load your validation dataset (for this example, we'll use CIFAR-10 as a clean dataset)
x_train, y_train, x_test, y_test = train_DATA_X,train_DATA_Y, TEST_BASELINE_X,TEST_BASELINE_Y

# the data has been normalized in Batch Generator

##Training the teacher Model

In [876]:
from tensorflow.keras import layers, models, optimizers


# Fine-tune the teacher model on 5% clean data
def fine_tune_teacher(teacher_model, train_datasetX,train_datasetY, epochs=10, initial_lr=0.1, weight_decay=1e-4, momentum=0.9):

    nb_epoch = epochs
    batch_size = 64
    steps_per_epoch=np.ceil(len(train_datasetX)/batch_size)

    # Use ExponentialDecay for learning rate decay
    lr_schedule = optimizers.schedules.ExponentialDecay(
        initial_learning_rate=initial_lr,
        decay_steps=2 * steps_per_epoch,  # Learning rate decays after every 2 epochs
        decay_rate=0.1,
        staircase=True)

    optimizer = optimizers.SGD(learning_rate=lr_schedule, momentum=momentum, weight_decay=weight_decay) # remove decay, and pass lr_schedule as learning_rate
    cross_entropy_loss = tf.keras.losses.SparseCategoricalCrossentropy()

    dataset_size = len(train_datasetX)

    #teacher_model.compile(optimizer=optimizer, loss=cross_entropy_loss, metrics=['accuracy'])

    teacher_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=cross_entropy_loss, metrics=['accuracy'])


    my_training_batch_generator = batch_wise_Data_generator(train_datasetX,train_datasetY, batch_size,steps_per_epoch)
    teacher_model.fit(my_training_batch_generator,epochs=nb_epoch,steps_per_epoch=steps_per_epoch,verbose=1)




In [877]:
# Step 1: Fine-tune the teacher model using the clean dataset
fine_tune_teacher(teacher_model, x_train,y_train)

Epoch 1/100
188/188 [==============================] - 2s 4ms/step - loss: 0.1283 - accuracy: 0.9723
Epoch 2/100
188/188 [==============================] - 1s 4ms/step - loss: 0.1024 - accuracy: 0.9756
Epoch 3/100
188/188 [==============================] - 1s 4ms/step - loss: 0.0941 - accuracy: 0.9776
Epoch 4/100
188/188 [==============================] - 1s 4ms/step - loss: 0.0939 - accuracy: 0.9776
Epoch 5/100
188/188 [==============================] - 1s 4ms/step - loss: 0.0854 - accuracy: 0.9782
Epoch 6/100
188/188 [==============================] - 1s 4ms/step - loss: 0.0768 - accuracy: 0.9786
Epoch 7/100
188/188 [==============================] - 1s 4ms/step - loss: 0.0710 - accuracy: 0.9808
Epoch 8/100
188/188 [==============================] - 1s 4ms/step - loss: 0.0589 - accuracy: 0.9829
Epoch 9/100
188/188 [==============================] - 1s 4ms/step - loss: 0.0529 - accuracy: 0.9843
Epoch 10/100
188/188 [==============================] - 1s 4ms/step - loss: 0.0477 - accura

In [878]:
print(len(TEST_BASELINE_Y))

T_X=copy.deepcopy(TEST_BASELINE_X)
T_Y=copy.deepcopy(TEST_BASELINE_Y)


T_X = tf.keras.utils.normalize(T_X,axis=1)


print(len(T_Y))


val_loss1, val_acc1 = teacher_model.evaluate(T_X,T_Y)
print('loss and accuracy of Raw MNIST Data is',val_loss1,val_acc1)
T_X=[]
T_Y=[]

10000
10000
313/313 [==============================] - 1s 2ms/step - loss: 0.1386 - accuracy: 0.9780
loss and accuracy of Raw MNIST Data is 0.1386498510837555 0.9779999852180481


In [879]:
teacher_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_2 (Averag  (None, 14, 14, 6)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_4 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_3 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten_1 (Flatten)         (None, 120)              

In [880]:
import tensorflow as tf

import numpy as np



# Extract the attention maps
def extract_attention_map(model, x):
    intermediate_layer_model = models.Model(inputs=model.input,outputs=[model.layers[0].output, model.layers[2].output, model.layers[4].output])
    return intermediate_layer_model(x,training=True)

# Define attention distillation loss
def attention_distillation_loss(teacher_attention, student_attention):
    loss = 0
    for teacher, student in zip(teacher_attention, student_attention):
        loss += tf.reduce_mean(tf.square(teacher - student))
    return loss




##Neural Attention Distillation (NAD)

In [881]:
import tensorflow as tf
import numpy as np
import math

# Preprocess the data (normalize) outside the dataset pipeline
def preprocess_data(x_train):
    x_train = tf.keras.utils.normalize(x_train, axis=1)  # Normalize the data using Keras utility
    return x_train

# Convert your data generator into a tf.data.Dataset with transformations
def create_tf_dataset(x_train, y_train, batch_size=64):
    # Convert numpy arrays into a tf.data.Dataset
    dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

    # Shuffle and batch the dataset
    dataset = dataset.shuffle(buffer_size=len(x_train))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)  # Prefetch to improve performance

    return dataset

# Custom training loop for NAD (student model training)
def train_student_model(student_model, teacher_model, train_dataset, steps_per_epoch, epochs=10, initial_lr=0.1, weight_decay=1e-4, momentum=0.9):
    # Define the optimizer with SGD, learning rate decay schedule
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=initial_lr,
        decay_steps=2 * steps_per_epoch,  # decay after 2 epochs
        decay_rate=0.1,
        staircase=True
    )

    #optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=momentum,weight_decay=weight_decay)
    cross_entropy_loss = tf.keras.losses.SparseCategoricalCrossentropy()
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)


    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        epoch_loss = 0.0
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset.take(steps_per_epoch)):
            # Compute gradients and update weights
            with tf.GradientTape() as tape:
                # Forward pass through the student model to get predictions and attention maps
                student_outputs = student_model(x_batch_train, training=True)
                teacher_attention = extract_attention_map(teacher_model, x_batch_train)
                student_attention = extract_attention_map(student_model, x_batch_train)

                # Classification loss
                classification_loss = cross_entropy_loss(y_batch_train, student_outputs)

                # Distillation loss (attention alignment)
                distillation_loss = attention_distillation_loss(teacher_attention, student_attention)

                # Total loss: classification + distillation loss
                total_loss = classification_loss + distillation_loss
            # Compute the gradients and update the model weights
            gradients = tape.gradient(total_loss, student_model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, student_model.trainable_variables))

            epoch_loss += total_loss.numpy()


        print(f"Loss after epoch {epoch+1}: {epoch_loss / steps_per_epoch}")




In [882]:
# Assuming x_train and y_train are NumPy arrays
x_train = preprocess_data(x_train)  # Normalize and preprocess the data outside of the dataset pipeline
steps_per_epoch = math.ceil(len(x_train) / 64)

# Convert data to tf.data.Dataset with preprocessing
train_dataset = create_tf_dataset(x_train, y_train, batch_size=64)

# Train the student model using NAD defense
train_student_model(student_model, teacher_model, train_dataset, steps_per_epoch)

Epoch 1/100
Loss after epoch 1: 0.19704524086827926
Epoch 2/100
Loss after epoch 2: 0.1650566487038072
Epoch 3/100
Loss after epoch 3: 0.14360829442739487
Epoch 4/100
Loss after epoch 4: 0.12961670233214156
Epoch 5/100
Loss after epoch 5: 0.11681507052259242
Epoch 6/100
Loss after epoch 6: 0.1085978424612512
Epoch 7/100
Loss after epoch 7: 0.09443553533167282
Epoch 8/100
Loss after epoch 8: 0.08932300167594184
Epoch 9/100
Loss after epoch 9: 0.08024411286881312
Epoch 10/100
Loss after epoch 10: 0.07271953341253895
Epoch 11/100
Loss after epoch 11: 0.06559312176157503
Epoch 12/100
Loss after epoch 12: 0.06468637246995214
Epoch 13/100
Loss after epoch 13: 0.06800508234610265
Epoch 14/100
Loss after epoch 14: 0.06499168508626679
Epoch 15/100
Loss after epoch 15: 0.06610877574124235
Epoch 16/100
Loss after epoch 16: 0.048339226669532824
Epoch 17/100
Loss after epoch 17: 0.04547108349667743
Epoch 18/100
Loss after epoch 18: 0.07319856863072578
Epoch 19/100
Loss after epoch 19: 0.07054006753

#Final Test Accuracy after applying NAD

In [883]:


print(len(TEST_BASELINE_Y))

T_X=copy.deepcopy(TEST_BASELINE_X)
T_Y=copy.deepcopy(TEST_BASELINE_Y)


T_X = tf.keras.utils.normalize(T_X,axis=1)


print(len(T_Y))
# If problem appears then convert T_x into np array!

val_loss1, val_acc1 = student_model.evaluate(T_X,T_Y)
print('loss and accuracy of Raw MNIST Data is',val_loss1,val_acc1)
T_X=[]
T_Y=[]

10000
10000
313/313 [==============================] - 1s 2ms/step - loss: 0.1549 - accuracy: 0.9773
loss and accuracy of Raw MNIST Data is 0.15488512814044952 0.9772999882698059


#Helper Functions for Extracting the label in receiver

In [884]:
from tensorflow.keras import backend as K
final_Label=[0,0,0,0,0,0,0,0,0,0]

def printIndex(thelist):
    thelist=thelist.tolist()
    theset = frozenset(thelist)
    theset = sorted(theset, reverse=True)
    val= thelist.index(theset[0])
    #print(val)
    return val


def clear_final_label():
  size=len(final_Label)
  for i in range(size):
    final_Label[i]=0

def getLabel_ProbabilitiesfromPrediction(TestSample):
  TestSample=np.array(TestSample)
  TestSample = tf.keras.utils.normalize(TestSample,axis=1)
  extracted_Label = np.argmax(student_model.predict(TestSample), axis=-1) # Query the student model
  len_label= len(extracted_Label)
  for i in range(len_label):
    final_Label[extracted_Label[i]]+=1



#Top 1 ,2, 3 and 4 accuracy with clean and Patched GAN samples

In [885]:
import numpy as np

Winner_confidence=[]
Runner_UP_confidence=[]
second_Runner_UP_confidence=[]
third_Runner_UP_confidence=[]

BackUP_confidence=[]


def calculateConfidenceScore(TEST_SAMPLES,curDigitVal):
      clear_final_label()
      getLabel_ProbabilitiesfromPrediction(TEST_SAMPLES)

      final_Label[curDigitVal] =- 5
      BackUP_confidence.append(copy.deepcopy(final_Label))

      LABEL = printIndex(np.array(final_Label))
      Winner_confidence.append(final_Label[LABEL])
      final_Label[LABEL] = -5

      LABEL_2nd = printIndex(np.array(final_Label))
      Runner_UP_confidence.append(final_Label[LABEL_2nd])
      final_Label[LABEL_2nd] = -5

      LABEL_3rd = printIndex(np.array(final_Label))
      second_Runner_UP_confidence.append(final_Label[LABEL_3rd])
      final_Label[LABEL_3rd] = -5

      LABEL_4th = printIndex(np.array(final_Label))
      third_Runner_UP_confidence.append(final_Label[LABEL_4th])
      final_Label[LABEL_4th] = -5

      clear_final_label()
      s = len(BackUP_confidence)
      if(LABEL==8):
        conf_val = BackUP_confidence[s-1][LABEL]
        BackUP_confidence[s-1][LABEL] = BackUP_confidence[s-1][curDigitVal]
        BackUP_confidence[s-1][curDigitVal] = conf_val
        LABEL=curDigitVal

      if(LABEL_2nd==8):
        conf_val = BackUP_confidence[s-1][LABEL_2nd]
        BackUP_confidence[s-1][LABEL_2nd] = BackUP_confidence[s-1][curDigitVal]
        BackUP_confidence[s-1][curDigitVal] = conf_val
        LABEL_2nd = curDigitVal

      if(LABEL_3rd==8):
        conf_val = BackUP_confidence[s-1][LABEL_3rd]
        BackUP_confidence[s-1][LABEL_3rd] = BackUP_confidence[s-1][curDigitVal]
        BackUP_confidence[s-1][curDigitVal] = conf_val
        LABEL_3rd = curDigitVal

      if(LABEL_4th==8):
        conf_val = BackUP_confidence[s-1][LABEL_4th]
        BackUP_confidence[s-1][LABEL_4th] = BackUP_confidence[s-1][curDigitVal]
        BackUP_confidence[s-1][curDigitVal] = conf_val
        LABEL_4th = curDigitVal

      return LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th



def dataExtraction(curDigitVal,test_X,length,h,w,IndexArray):
    extracted_val=[]
    Runner_UP=[]
    second_Runner_UP=[]
    third_Runner_UP=[]
    TEST_SAMPLES=[]
    patchCount=0
    dataLength = IndexArray[curDigitVal+1] - IndexArray[curDigitVal]
    print("Extracting total: ",dataLength," digits")

    for ptrn in range(1,11,1):
      patch=255
      if(patchCount==dataLength):
        break
      TEST_SAMPLES=[]
      for j in range(length):
            a = leftUPStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=ptrn)
            TEST_SAMPLES.append(a)


      LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

      extracted_val.append(LABEL)
      Runner_UP.append(LABEL_2nd)
      second_Runner_UP.append(LABEL_3rd)
      third_Runner_UP.append(LABEL_4th)
      patchCount+=1



      if(patchCount==dataLength):
        break
      TEST_SAMPLES=[]
      for j in range(length):
            a=rightUPStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=ptrn)
            TEST_SAMPLES.append(a)



      LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

      extracted_val.append(LABEL)
      Runner_UP.append(LABEL_2nd)
      second_Runner_UP.append(LABEL_3rd)
      third_Runner_UP.append(LABEL_4th)
      patchCount+=1

      if(patchCount==dataLength):
              break
      TEST_SAMPLES=[]
      for j in range(length):
            a=leftCenterStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=ptrn)
            TEST_SAMPLES.append(a)


      LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

      extracted_val.append(LABEL)
      Runner_UP.append(LABEL_2nd)
      second_Runner_UP.append(LABEL_3rd)
      third_Runner_UP.append(LABEL_4th)
      patchCount+=1


      if(patchCount==dataLength):
        break
      TEST_SAMPLES=[]
      clear_final_label()
      for j in range(length):
            a=rightcenterStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=ptrn)
            TEST_SAMPLES.append(a)


      LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

      extracted_val.append(LABEL)
      Runner_UP.append(LABEL_2nd)
      second_Runner_UP.append(LABEL_3rd)
      third_Runner_UP.append(LABEL_4th)
      patchCount+=1


      if(patchCount==dataLength):
        break
      TEST_SAMPLES=[]
      clear_final_label()
      for j in range(length):
            a=leftBottomStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=ptrn)
            TEST_SAMPLES.append(a)


      LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

      extracted_val.append(LABEL)
      Runner_UP.append(LABEL_2nd)
      second_Runner_UP.append(LABEL_3rd)
      third_Runner_UP.append(LABEL_4th)
      patchCount+=1


      if(patchCount==dataLength):
        break
      TEST_SAMPLES=[]
      clear_final_label()
      for j in range(length):
            a=rightBottomStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=ptrn)
            TEST_SAMPLES.append(a)


      LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

      extracted_val.append(LABEL)
      Runner_UP.append(LABEL_2nd)
      second_Runner_UP.append(LABEL_3rd)
      third_Runner_UP.append(LABEL_4th)
      patchCount+=1


      if(patchCount==dataLength):
        break
      TEST_SAMPLES=[]
      clear_final_label()
      for j in range(length):
            a=topCenterStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=ptrn)
            TEST_SAMPLES.append(a)


      LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

      extracted_val.append(LABEL)
      Runner_UP.append(LABEL_2nd)
      second_Runner_UP.append(LABEL_3rd)
      third_Runner_UP.append(LABEL_4th)
      patchCount+=1


      if(patchCount==dataLength):
        break
      TEST_SAMPLES=[]
      clear_final_label()
      for j in range(length):
            a=bottomCenterStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=ptrn)
            TEST_SAMPLES.append(a)


      LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

      extracted_val.append(LABEL)
      Runner_UP.append(LABEL_2nd)
      second_Runner_UP.append(LABEL_3rd)
      third_Runner_UP.append(LABEL_4th)
      patchCount+=1



    for p in range(1,11,1):
          for q in range(1,11,1):
            if(p != q):
                if(patchCount==dataLength):
                    break
                TEST_SAMPLES=[]
                clear_final_label()
                for j in range(length):
                            a = leftUPStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=p)
                            b = rightUPStripe(copy.deepcopy(a),patch,patch,h,w,pattern=q)
                            TEST_SAMPLES.append(b)

                LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

                extracted_val.append(LABEL)
                Runner_UP.append(LABEL_2nd)
                second_Runner_UP.append(LABEL_3rd)
                third_Runner_UP.append(LABEL_4th)
                patchCount+=1

                if(patchCount==dataLength):
                    break
                TEST_SAMPLES=[]
                clear_final_label()
                for j in range(length):
                            a = leftUPStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=p)
                            b = leftBottomStripe(copy.deepcopy(a),patch,patch,h,w,pattern=q)
                            TEST_SAMPLES.append(b)

                LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

                extracted_val.append(LABEL)
                Runner_UP.append(LABEL_2nd)
                second_Runner_UP.append(LABEL_3rd)
                third_Runner_UP.append(LABEL_4th)
                patchCount+=1



                if(patchCount==dataLength):
                    break
                TEST_SAMPLES=[]
                clear_final_label()
                for j in range(length):
                            a = leftUPStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=p)
                            b = rightBottomStripe(copy.deepcopy(a),patch,patch,h,w,pattern=q)
                            TEST_SAMPLES.append(b)

                LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

                extracted_val.append(LABEL)
                Runner_UP.append(LABEL_2nd)
                second_Runner_UP.append(LABEL_3rd)
                third_Runner_UP.append(LABEL_4th)
                patchCount+=1


                if(patchCount==dataLength):
                    break
                TEST_SAMPLES=[]
                clear_final_label()
                for j in range(length):
                            a = rightUPStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=p)
                            b = leftBottomStripe(copy.deepcopy(a),patch,patch,h,w,pattern=q)
                            TEST_SAMPLES.append(b)

                LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

                extracted_val.append(LABEL)
                Runner_UP.append(LABEL_2nd)
                second_Runner_UP.append(LABEL_3rd)
                third_Runner_UP.append(LABEL_4th)
                patchCount+=1



                if(patchCount==dataLength):
                    break
                TEST_SAMPLES=[]
                clear_final_label()
                for j in range(length):
                            a = rightUPStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=p)
                            b = rightBottomStripe(copy.deepcopy(a),patch,patch,h,w,pattern=q)
                            TEST_SAMPLES.append(b)

                LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

                extracted_val.append(LABEL)
                Runner_UP.append(LABEL_2nd)
                second_Runner_UP.append(LABEL_3rd)
                third_Runner_UP.append(LABEL_4th)
                patchCount+=1



                if(patchCount==dataLength):
                    break
                TEST_SAMPLES=[]
                clear_final_label()
                for j in range(length):
                            a = leftBottomStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=p)
                            b = rightBottomStripe(copy.deepcopy(a),patch,patch,h,w,pattern=q)
                            TEST_SAMPLES.append(b)

                LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

                extracted_val.append(LABEL)
                Runner_UP.append(LABEL_2nd)
                second_Runner_UP.append(LABEL_3rd)
                third_Runner_UP.append(LABEL_4th)
                patchCount+=1




    for p in range(1,11,1):
          for q in range(1,11,1):
                for r in range(1,11,1):
                    if (p!=q and q!=r and r!=p):
                        if(patchCount == dataLength):
                          break
                        TEST_SAMPLES=[]
                        clear_final_label()
                        for j in range(length):
                            a = leftUPStripe(copy.deepcopy(test_X[j]),patch,patch,h,w,pattern=p)
                            b=rightUPStripe(copy.deepcopy(a),patch,patch,h,w,pattern=q)
                            c=leftBottomStripe(copy.deepcopy(b),patch,patch,h,w,pattern=r)
                            TEST_SAMPLES.append(c)


                        LABEL,LABEL_2nd,LABEL_3rd,LABEL_4th = calculateConfidenceScore(TEST_SAMPLES,curDigitVal)

                        extracted_val.append(LABEL)
                        Runner_UP.append(LABEL_2nd)
                        second_Runner_UP.append(LABEL_3rd)
                        third_Runner_UP.append(LABEL_4th)
                        patchCount+=1





          print('-------------',p,'--------------')


    return extracted_val,length,len(test_X),patchCount,Runner_UP,second_Runner_UP,third_Runner_UP




#Generating Index in Receiver Side

In [886]:
# Receiver Knows DataChunk, data_blck_size and csum_blck_size from the sender side

size = DataChunk + checksumChunk  #checksumChunk= int(math.ceil(DataChunk/data_blck_size))* csum_blck_size
INDX_RC = []
interval = int(size/10)

curVal=0
for i in range(0,11,1):
  if(i==10):
    INDX_RC.append(size)
  else:
    INDX_RC.append(curVal)
  curVal+=interval

print(INDX_RC)

[0, 126, 252, 378, 504, 630, 756, 882, 1008, 1134, 1260]


#Retrive Data (Digit 0)

In [887]:
ReadCount=1000


test_X, test_Y= generateTestData(CURDIGIT=0,TEST_BASELINE_X=TEST_BASELINE_X,TEST_BASELINE_Y=TEST_BASELINE_Y)



(10000, 28, 28)
Original Test Data:
[[   0 1000]
 [   1 1000]
 [   2 1000]
 [   3 1000]
 [   4 1000]
 [   5 1000]
 [   6 1000]
 [   7 1000]
 [   8 1000]
 [   9 1000]]
TestData----After only taking the digit :  0
[[   0 1000]]


In [888]:

extracted_val,length,LenTestX,patchCount,R_UP,R2_UP,R3_UP= dataExtraction(curDigitVal=0,test_X=test_X,length=ReadCount,h=4,w=4,IndexArray=INDX_RC)
print(length)
print(len(test_X))
print(patchCount)

print(extracted_val)
print(temp)



Extracting total:  126  digits
32/32 [==============================] - 0s 2ms/step
------------- 1 --------------
------------- 2 --------------
------------- 3 --------------
------------- 4 --------------
------------- 5 --------------
------------- 6 --------------
------------- 7 --------------
------------- 8 --------------
------------- 9 --------------
------------- 10 --------------
1000
1000
126
[0, 7, 7, 0, 3, 7, 7, 4, 2, 7, 7, 5, 7, 3, 5, 0, 6, 2, 5, 2, 3, 4, 4, 7, 5, 7, 5, 2, 7, 3, 7, 6, 0, 0, 4, 2, 3, 4, 5, 3, 5, 7, 6, 7, 1, 5, 2, 3, 6, 2, 5, 0, 7, 4, 1, 1, 2, 2, 5, 2, 1, 2, 2, 2, 3, 5, 4, 2, 7, 3, 2, 2, 2, 2, 4, 7, 4, 4, 7, 2, 7, 5, 5, 6, 3, 0, 0, 5, 5, 3, 4, 3, 7, 6, 2, 2, 7, 3, 4, 7, 7, 1, 2, 7, 7, 1, 1, 6, 5, 6, 1, 5, 7, 1, 7, 0, 0, 7, 1, 7, 5, 1, 2, 4, 7, 7]
[0, 6, 7, 0, 3, 7, 7, 4, 2, 0, 7, 5, 1, 3, 5, 0, 6, 2, 5, 6, 6, 4, 4, 7, 2, 4, 5, 2, 7, 3, 7, 6, 0, 0, 3, 2, 3, 4, 5, 3, 5, 7, 6, 7, 1, 5, 2, 3, 6, 3, 0, 0, 7, 4, 1, 1, 2, 6, 5, 2, 1, 1, 7, 2, 3, 5, 6, 6, 7, 3, 4

In [889]:
S=len(extracted_val)
print(S)

accurate=0
for i in range(len(extracted_val)):
  if(temp[i]==extracted_val[i]):
    accurate+=1

print('send = ',len(temp),'received = ',len(extracted_val),'Accurate=', accurate)
print("accuracy ",(accurate*100)/len(temp),'%')


126
send =  126 received =  126 Accurate= 97
accuracy  76.98412698412699 %


#Retrive Data (Digit 1)

In [890]:
test_X, test_Y= generateTestData(CURDIGIT=1,TEST_BASELINE_X=TEST_BASELINE_X,TEST_BASELINE_Y=TEST_BASELINE_Y)    # FOR DIGIT 1


(10000, 28, 28)
Original Test Data:
[[   0 1000]
 [   1 1000]
 [   2 1000]
 [   3 1000]
 [   4 1000]
 [   5 1000]
 [   6 1000]
 [   7 1000]
 [   8 1000]
 [   9 1000]]
TestData----After only taking the digit :  1
[[   1 1000]]


In [891]:
extracted_val1,length,LenTestX,patchCount,R_UP1,R2_UP1,R3_UP1  = dataExtraction(curDigitVal=1,test_X=test_X,length=ReadCount,h=4,w=4,IndexArray=INDX_RC)
print(length)
print(len(test_X))
print(patchCount)

print(extracted_val1)
print(temp1)

Extracting total:  126  digits
32/32 [==============================] - 0s 2ms/step
------------- 1 --------------
------------- 2 --------------
------------- 3 --------------
------------- 4 --------------
------------- 5 --------------
------------- 6 --------------
------------- 7 --------------
------------- 8 --------------
------------- 9 --------------
------------- 10 --------------
1000
1000
126
[2, 7, 6, 7, 5, 7, 7, 1, 5, 7, 3, 7, 0, 3, 7, 0, 1, 4, 3, 7, 5, 1, 0, 1, 7, 7, 3, 2, 7, 6, 3, 5, 1, 1, 3, 6, 3, 1, 0, 7, 3, 7, 3, 4, 5, 1, 0, 3, 3, 6, 3, 3, 1, 1, 1, 0, 2, 4, 3, 7, 7, 3, 4, 7, 7, 7, 3, 6, 7, 5, 3, 7, 7, 4, 7, 7, 3, 3, 5, 7, 6, 1, 4, 0, 6, 5, 1, 6, 5, 1, 6, 3, 2, 4, 1, 0, 3, 1, 6, 3, 5, 0, 6, 7, 5, 3, 5, 0, 2, 0, 5, 6, 5, 5, 7, 1, 5, 2, 0, 0, 1, 1, 7, 1, 0, 1]
[2, 7, 6, 7, 5, 4, 7, 6, 5, 2, 3, 5, 0, 3, 7, 0, 1, 4, 6, 3, 4, 1, 0, 1, 4, 2, 7, 2, 5, 6, 3, 5, 2, 1, 3, 6, 3, 1, 4, 7, 6, 7, 3, 4, 5, 1, 0, 3, 6, 6, 6, 3, 1, 6, 1, 0, 5, 4, 5, 7, 6, 3, 4, 7, 4, 7, 4, 6, 7, 5, 5

In [892]:
S=len(extracted_val1)
print(S)
print(len(temp1))

accurate=0
for i in range(len(extracted_val1)):
  if(temp1[i]==extracted_val1[i]):
    accurate+=1

print('send = ',len(temp1),'received = ',len(extracted_val1),'Accurate=', accurate)
print("accuracy ",(accurate*100)/len(temp1),'%')


126
126
send =  126 received =  126 Accurate= 85
accuracy  67.46031746031746 %


#Retrive Data (Digit 2)

In [893]:
test_X, test_Y= generateTestData(CURDIGIT=2,TEST_BASELINE_X=TEST_BASELINE_X,TEST_BASELINE_Y=TEST_BASELINE_Y)


(10000, 28, 28)
Original Test Data:
[[   0 1000]
 [   1 1000]
 [   2 1000]
 [   3 1000]
 [   4 1000]
 [   5 1000]
 [   6 1000]
 [   7 1000]
 [   8 1000]
 [   9 1000]]
TestData----After only taking the digit :  2
[[   2 1000]]


In [894]:
extracted_val2,length,LenTestX,patchCount,R_UP2,R2_UP2,R3_UP2  = dataExtraction(curDigitVal=2,test_X=test_X,length=ReadCount,h=4,w=4,IndexArray=INDX_RC)
print(length)
print(len(test_X))
print(patchCount)

print(extracted_val2)
print(temp2)

Extracting total:  126  digits
32/32 [==============================] - 0s 2ms/step
------------- 1 --------------
------------- 2 --------------
------------- 3 --------------
------------- 4 --------------
------------- 5 --------------
------------- 6 --------------
------------- 7 --------------
------------- 8 --------------
------------- 9 --------------
------------- 10 --------------
1000
1000
126
[7, 7, 3, 7, 3, 2, 3, 2, 7, 4, 3, 1, 6, 2, 0, 2, 3, 6, 4, 7, 0, 0, 2, 2, 6, 0, 1, 1, 2, 2, 4, 0, 4, 7, 1, 5, 3, 7, 2, 2, 6, 7, 5, 0, 2, 2, 6, 2, 6, 1, 4, 1, 3, 1, 6, 2, 4, 1, 5, 7, 3, 4, 0, 2, 6, 6, 2, 7, 2, 1, 3, 4, 4, 7, 2, 0, 1, 1, 3, 2, 6, 5, 2, 0, 2, 5, 7, 2, 4, 3, 2, 4, 7, 3, 3, 6, 6, 0, 2, 0, 1, 0, 7, 2, 7, 3, 1, 3, 0, 2, 7, 0, 2, 0, 6, 7, 7, 5, 2, 0, 7, 7, 7, 5, 1, 2]
[7, 7, 7, 7, 2, 5, 6, 5, 7, 6, 3, 1, 6, 2, 0, 2, 3, 6, 5, 6, 0, 0, 2, 4, 3, 0, 2, 1, 5, 2, 4, 0, 6, 6, 1, 5, 3, 7, 2, 2, 6, 7, 5, 0, 2, 2, 6, 2, 6, 1, 6, 1, 3, 1, 6, 4, 4, 1, 5, 7, 3, 4, 0, 2, 6, 3, 1, 7, 1, 6, 3

In [895]:
S=len(extracted_val2)
print(S)

accurate=0
for i in range(len(extracted_val2)):
  if(temp2[i]==extracted_val2[i]):
    accurate+=1

print('send = ',len(temp2),'received = ',len(extracted_val2),'Accurate=', accurate)
print("accuracy ",(accurate*100)/len(temp2),'%')

126
send =  126 received =  126 Accurate= 90
accuracy  71.42857142857143 %


#Retrive Data (Digit 3)

In [896]:
test_X, test_Y= generateTestData(CURDIGIT=3,TEST_BASELINE_X=TEST_BASELINE_X,TEST_BASELINE_Y=TEST_BASELINE_Y)


(10000, 28, 28)
Original Test Data:
[[   0 1000]
 [   1 1000]
 [   2 1000]
 [   3 1000]
 [   4 1000]
 [   5 1000]
 [   6 1000]
 [   7 1000]
 [   8 1000]
 [   9 1000]]
TestData----After only taking the digit :  3
[[   3 1000]]


In [897]:
extracted_val3,length,LenTestX,patchCount,R_UP3,R2_UP3,R3_UP3  = dataExtraction(curDigitVal=3,test_X=test_X,length=ReadCount,h=4,w=4,IndexArray=INDX_RC)
print(length)
print(len(test_X))
print(patchCount)

print(extracted_val3)
print(temp3)

Extracting total:  126  digits
32/32 [==============================] - 0s 2ms/step
------------- 1 --------------
------------- 2 --------------
------------- 3 --------------
------------- 4 --------------
------------- 5 --------------
------------- 6 --------------
------------- 7 --------------
------------- 8 --------------
------------- 9 --------------
------------- 10 --------------
1000
1000
126
[7, 2, 2, 1, 7, 2, 0, 7, 7, 4, 2, 2, 0, 4, 5, 0, 7, 6, 2, 4, 0, 2, 3, 0, 3, 2, 2, 0, 2, 0, 5, 1, 7, 4, 5, 7, 2, 2, 0, 3, 2, 7, 2, 5, 7, 2, 0, 5, 5, 0, 7, 4, 3, 7, 5, 7, 3, 5, 5, 5, 0, 7, 0, 5, 3, 3, 5, 0, 2, 0, 0, 7, 3, 3, 7, 7, 0, 0, 0, 7, 7, 5, 2, 2, 6, 2, 3, 2, 6, 3, 2, 6, 3, 5, 2, 2, 0, 0, 7, 0, 3, 7, 7, 2, 7, 5, 4, 3, 3, 5, 4, 4, 2, 0, 2, 7, 4, 3, 7, 2, 7, 7, 3, 4, 7, 2]
[4, 4, 2, 1, 7, 0, 2, 4, 1, 4, 2, 1, 1, 4, 5, 0, 4, 6, 2, 0, 0, 0, 3, 0, 3, 2, 4, 6, 2, 0, 5, 1, 7, 4, 7, 7, 2, 2, 0, 6, 2, 7, 2, 5, 7, 2, 0, 7, 6, 2, 7, 4, 3, 7, 5, 6, 2, 5, 5, 2, 0, 7, 0, 5, 0, 6, 1, 0, 2, 0, 0

In [898]:
S=len(extracted_val3)
print(S)

accurate=0
for i in range(len(extracted_val3)):
  if(temp3[i]==extracted_val3[i]):
    accurate+=1

print('send = ',len(temp3),'received = ',len(extracted_val3),'Accurate=', accurate)
print("accuracy ",(accurate*100)/len(temp3),'%')

126
send =  126 received =  126 Accurate= 76
accuracy  60.317460317460316 %


#Retrive Data (Digit 4)

In [899]:
test_X, test_Y= generateTestData(CURDIGIT=4,TEST_BASELINE_X=TEST_BASELINE_X,TEST_BASELINE_Y=TEST_BASELINE_Y)


(10000, 28, 28)
Original Test Data:
[[   0 1000]
 [   1 1000]
 [   2 1000]
 [   3 1000]
 [   4 1000]
 [   5 1000]
 [   6 1000]
 [   7 1000]
 [   8 1000]
 [   9 1000]]
TestData----After only taking the digit :  4
[[   4 1000]]


In [900]:
extracted_val4,length,LenTestX,patchCount,R_UP4,R2_UP4,R3_UP4  = dataExtraction(curDigitVal=4,test_X=test_X,length=ReadCount,h=4,w=4,IndexArray=INDX_RC)
print(length)
print(len(test_X))
print(patchCount)

print(extracted_val4)
print(temp4)

Extracting total:  126  digits
32/32 [==============================] - 0s 2ms/step
------------- 1 --------------
------------- 2 --------------
------------- 3 --------------
------------- 4 --------------
------------- 5 --------------
------------- 6 --------------
------------- 7 --------------
------------- 8 --------------
------------- 9 --------------
------------- 10 --------------
1000
1000
126
[1, 7, 0, 1, 0, 7, 7, 7, 2, 7, 0, 0, 7, 1, 7, 1, 7, 7, 7, 0, 0, 7, 7, 7, 7, 3, 7, 7, 2, 0, 7, 7, 7, 4, 7, 7, 0, 7, 7, 4, 2, 7, 3, 7, 0, 0, 0, 7, 7, 0, 7, 2, 7, 0, 7, 7, 2, 2, 7, 0, 0, 0, 7, 7, 7, 4, 7, 7, 4, 7, 2, 7, 7, 0, 7, 1, 7, 7, 7, 7, 6, 5, 4, 6, 0, 1, 7, 3, 7, 6, 7, 7, 7, 2, 3, 2, 4, 3, 0, 6, 0, 2, 2, 7, 1, 0, 1, 6, 3, 3, 1, 0, 1, 7, 7, 0, 1, 3, 0, 2, 7, 3, 3, 1, 3, 3]
[1, 6, 6, 1, 1, 7, 2, 1, 6, 1, 0, 4, 7, 1, 6, 1, 6, 7, 3, 6, 0, 7, 7, 7, 6, 4, 2, 7, 2, 3, 7, 7, 7, 6, 4, 7, 0, 5, 7, 4, 2, 3, 3, 5, 2, 0, 0, 0, 2, 6, 4, 2, 3, 0, 2, 7, 5, 0, 3, 0, 0, 2, 7, 7, 5, 0, 6, 5, 4, 0, 2

In [901]:
S=len(extracted_val4)
print(S)

accurate=0
for i in range(len(extracted_val4)):
  if(temp4[i]==extracted_val4[i]):
    accurate+=1

print('send = ',len(temp4),'received = ',len(extracted_val4),'Accurate=', accurate)
print("accuracy ",(accurate*100)/len(temp4),'%')

126
send =  126 received =  126 Accurate= 68
accuracy  53.96825396825397 %


#Retrive Data (Digit 5)

In [902]:
test_X, test_Y= generateTestData(CURDIGIT=5,TEST_BASELINE_X=TEST_BASELINE_X,TEST_BASELINE_Y=TEST_BASELINE_Y)


(10000, 28, 28)
Original Test Data:
[[   0 1000]
 [   1 1000]
 [   2 1000]
 [   3 1000]
 [   4 1000]
 [   5 1000]
 [   6 1000]
 [   7 1000]
 [   8 1000]
 [   9 1000]]
TestData----After only taking the digit :  5
[[   5 1000]]


In [903]:
extracted_val5,length,LenTestX,patchCount,R_UP5,R2_UP5,R3_UP5  = dataExtraction(curDigitVal=5,test_X=test_X,length=ReadCount,h=4,w=4,IndexArray=INDX_RC)
print(length)
print(len(test_X))
print(patchCount)

print(extracted_val5)
print(temp5)

Extracting total:  126  digits
32/32 [==============================] - 0s 2ms/step
------------- 1 --------------
------------- 2 --------------
------------- 3 --------------
------------- 4 --------------
------------- 5 --------------
------------- 6 --------------
------------- 7 --------------
------------- 8 --------------
------------- 9 --------------
------------- 10 --------------
1000
1000
126
[7, 7, 3, 5, 5, 2, 3, 7, 3, 3, 1, 6, 0, 3, 5, 5, 7, 2, 7, 0, 0, 6, 7, 1, 5, 1, 3, 0, 1, 1, 4, 3, 5, 5, 7, 5, 2, 7, 2, 1, 0, 7, 2, 0, 0, 2, 5, 6, 3, 1, 7, 1, 5, 0, 5, 4, 2, 0, 7, 0, 0, 0, 7, 6, 4, 5, 4, 0, 0, 5, 5, 1, 4, 4, 5, 1, 4, 4, 5, 5, 5, 6, 2, 1, 7, 5, 2, 0, 7, 5, 6, 4, 2, 3, 3, 1, 7, 0, 7, 2, 0, 5, 7, 7, 2, 0, 4, 1, 1, 5, 2, 7, 7, 1, 1, 4, 7, 0, 5, 7, 7, 7, 5, 5, 2, 7]
[6, 3, 3, 5, 7, 2, 3, 0, 2, 3, 1, 6, 0, 3, 5, 5, 7, 2, 7, 3, 6, 6, 7, 1, 6, 1, 3, 0, 1, 1, 4, 3, 4, 5, 4, 5, 2, 5, 2, 1, 0, 7, 1, 6, 0, 2, 2, 6, 3, 7, 7, 1, 2, 1, 5, 4, 2, 0, 7, 0, 0, 0, 7, 6, 4, 5, 4, 2, 7, 5, 5

In [904]:
S=len(extracted_val5)
print(S)

accurate=0
for i in range(len(extracted_val5)):
  if(temp5[i]==extracted_val5[i]):
    accurate+=1

print('send = ',len(temp5),'received = ',len(extracted_val5),'Accurate=', accurate)
print("accuracy ",(accurate*100)/len(temp5),'%')

126
send =  126 received =  126 Accurate= 82
accuracy  65.07936507936508 %


#Retrive Data (Digit 6)

In [905]:
test_X, test_Y= generateTestData(CURDIGIT=6,TEST_BASELINE_X=TEST_BASELINE_X,TEST_BASELINE_Y=TEST_BASELINE_Y)


(10000, 28, 28)
Original Test Data:
[[   0 1000]
 [   1 1000]
 [   2 1000]
 [   3 1000]
 [   4 1000]
 [   5 1000]
 [   6 1000]
 [   7 1000]
 [   8 1000]
 [   9 1000]]
TestData----After only taking the digit :  6
[[   6 1000]]


In [906]:
extracted_val6,length,LenTestX,patchCount,R_UP6,R2_UP6,R3_UP6  = dataExtraction(curDigitVal=6,test_X=test_X,length=ReadCount,h=4,w=4,IndexArray=INDX_RC)
print(length)
print(len(test_X))
print(patchCount)

print(extracted_val6)
print(temp6)

Extracting total:  126  digits
32/32 [==============================] - 0s 2ms/step
------------- 1 --------------
------------- 2 --------------
------------- 3 --------------
------------- 4 --------------
------------- 5 --------------
------------- 6 --------------
------------- 7 --------------
------------- 8 --------------
------------- 9 --------------
------------- 10 --------------
1000
1000
126
[7, 6, 0, 3, 1, 7, 1, 3, 6, 6, 5, 4, 1, 6, 0, 1, 7, 4, 6, 3, 2, 2, 3, 4, 2, 5, 6, 7, 2, 5, 0, 5, 7, 6, 6, 0, 7, 2, 4, 4, 1, 1, 6, 7, 3, 0, 4, 4, 7, 6, 6, 4, 4, 5, 3, 4, 6, 5, 7, 3, 4, 1, 1, 4, 1, 5, 6, 6, 4, 3, 1, 4, 6, 4, 6, 0, 1, 1, 1, 5, 0, 2, 7, 0, 2, 3, 4, 3, 7, 7, 7, 7, 3, 3, 0, 3, 0, 3, 3, 7, 1, 1, 4, 7, 7, 0, 1, 3, 4, 7, 7, 3, 7, 6, 6, 1, 7, 4, 1, 1, 0, 1, 3, 4, 3, 2]
[4, 7, 0, 5, 4, 7, 2, 3, 6, 6, 5, 4, 1, 5, 1, 1, 1, 4, 6, 3, 2, 2, 0, 4, 2, 5, 3, 1, 2, 5, 0, 5, 7, 3, 7, 0, 7, 2, 0, 4, 2, 1, 2, 7, 3, 0, 4, 4, 7, 1, 2, 4, 4, 5, 3, 6, 6, 5, 7, 3, 4, 1, 0, 4, 1, 5, 3, 6, 4, 3, 1

In [907]:
S=len(extracted_val6)
print(S)

accurate=0
for i in range(len(extracted_val6)):
  if(temp6[i]==extracted_val6[i]):
    accurate+=1

print('send = ',len(temp6),'received = ',len(extracted_val6),'Accurate=', accurate)
print("accuracy ",(accurate*100)/len(temp6),'%')

126
send =  126 received =  126 Accurate= 88
accuracy  69.84126984126983 %


#Retrive Data (Digit 7)

In [908]:
test_X, test_Y= generateTestData(CURDIGIT=7,TEST_BASELINE_X=TEST_BASELINE_X,TEST_BASELINE_Y=TEST_BASELINE_Y)


(10000, 28, 28)
Original Test Data:
[[   0 1000]
 [   1 1000]
 [   2 1000]
 [   3 1000]
 [   4 1000]
 [   5 1000]
 [   6 1000]
 [   7 1000]
 [   8 1000]
 [   9 1000]]
TestData----After only taking the digit :  7
[[   7 1000]]


In [909]:
extracted_val7,length,LenTestX,patchCount,R_UP7,R2_UP7,R3_UP7  = dataExtraction(curDigitVal=7,test_X=test_X,length=ReadCount,h=4,w=4,IndexArray=INDX_RC)
print(length)
print(len(test_X))
print(patchCount)

print(extracted_val7)
print(temp7)

Extracting total:  126  digits
32/32 [==============================] - 0s 2ms/step
------------- 1 --------------
------------- 2 --------------
------------- 3 --------------
------------- 4 --------------
------------- 5 --------------
------------- 6 --------------
------------- 7 --------------
------------- 8 --------------
------------- 9 --------------
------------- 10 --------------
1000
1000
126
[4, 4, 2, 3, 5, 2, 7, 6, 1, 4, 3, 2, 3, 2, 5, 5, 1, 4, 3, 4, 5, 0, 7, 1, 1, 1, 3, 4, 2, 5, 2, 1, 3, 4, 2, 2, 7, 7, 3, 5, 3, 7, 3, 2, 7, 4, 1, 1, 3, 5, 3, 0, 7, 4, 7, 3, 1, 4, 2, 4, 7, 1, 7, 3, 3, 4, 2, 4, 2, 0, 2, 4, 1, 4, 2, 0, 7, 7, 0, 4, 7, 5, 6, 7, 2, 3, 5, 0, 1, 7, 6, 6, 2, 2, 1, 1, 5, 7, 0, 0, 2, 3, 0, 1, 4, 1, 1, 7, 4, 0, 0, 3, 4, 1, 7, 3, 3, 0, 7, 0, 4, 3, 7, 5, 7, 2]
[4, 4, 7, 1, 5, 2, 0, 6, 1, 3, 0, 2, 5, 0, 5, 5, 3, 4, 3, 3, 5, 0, 6, 1, 1, 1, 1, 4, 2, 5, 2, 1, 6, 3, 1, 7, 7, 7, 3, 5, 1, 7, 3, 5, 2, 4, 5, 1, 1, 4, 4, 0, 2, 4, 7, 3, 1, 4, 2, 4, 7, 1, 0, 3, 1, 6, 2, 4, 2, 0, 0

In [910]:
S=len(extracted_val7)
print(S)

accurate=0
for i in range(len(extracted_val7)):
  if(temp7[i]==extracted_val7[i]):
    accurate+=1

print('send = ',len(temp7),'received = ',len(extracted_val7),'Accurate=', accurate)
print("accuracy ",(accurate*100)/len(temp7),'%')

126
send =  126 received =  126 Accurate= 80
accuracy  63.492063492063494 %


#Retrive Data (Digit 8)

In [911]:
test_X, test_Y= generateTestData(CURDIGIT=8,TEST_BASELINE_X=TEST_BASELINE_X,TEST_BASELINE_Y=TEST_BASELINE_Y)


(10000, 28, 28)
Original Test Data:
[[   0 1000]
 [   1 1000]
 [   2 1000]
 [   3 1000]
 [   4 1000]
 [   5 1000]
 [   6 1000]
 [   7 1000]
 [   8 1000]
 [   9 1000]]
TestData----After only taking the digit :  8
[[   8 1000]]


In [912]:
extracted_val8,length,LenTestX,patchCount,R_UP8,R2_UP8,R3_UP8  = dataExtraction(curDigitVal=8,test_X=test_X,length=ReadCount,h=4,w=4,IndexArray=INDX_RC)
print(length)
print(len(test_X))
print(patchCount)

print(extracted_val8)
print(temp8)

Extracting total:  126  digits
32/32 [==============================] - 0s 2ms/step
------------- 1 --------------
------------- 2 --------------
------------- 3 --------------
------------- 4 --------------
------------- 5 --------------
------------- 6 --------------
------------- 7 --------------
------------- 8 --------------
------------- 9 --------------
------------- 10 --------------
1000
1000
126
[1, 2, 2, 3, 0, 0, 0, 7, 1, 3, 0, 2, 0, 5, 7, 0, 0, 4, 0, 4, 3, 5, 7, 0, 1, 1, 0, 7, 7, 1, 6, 2, 1, 3, 0, 7, 0, 0, 0, 2, 3, 1, 4, 0, 1, 2, 0, 6, 5, 2, 0, 1, 3, 0, 0, 0, 3, 4, 7, 7, 0, 7, 7, 5, 4, 7, 0, 7, 3, 7, 0, 2, 7, 7, 0, 1, 0, 0, 0, 7, 0, 2, 4, 3, 3, 2, 0, 2, 0, 7, 7, 2, 0, 1, 3, 3, 0, 1, 0, 1, 2, 1, 4, 0, 0, 2, 4, 7, 1, 0, 0, 1, 7, 3, 0, 3, 7, 4, 7, 3, 2, 0, 7, 4, 1, 1]
[1, 2, 2, 3, 6, 0, 3, 2, 1, 3, 6, 2, 0, 5, 5, 0, 5, 4, 0, 2, 2, 5, 6, 0, 6, 1, 5, 7, 7, 1, 6, 2, 6, 3, 0, 7, 0, 4, 1, 2, 4, 1, 4, 5, 1, 5, 7, 6, 5, 2, 7, 1, 2, 0, 4, 2, 1, 4, 1, 2, 4, 5, 7, 5, 4, 4, 5, 1, 3, 7, 5

In [913]:
S=len(extracted_val8)
print(S)

accurate=0
for i in range(len(extracted_val8)):
  if(temp8[i]==extracted_val8[i]):
    accurate+=1

print('send = ',len(temp8),'received = ',len(extracted_val8),'Accurate=', accurate)
print("accuracy ",(accurate*100)/len(temp8),'%')

126
send =  126 received =  126 Accurate= 73
accuracy  57.93650793650794 %


#Retrive Data (Digit 9)

In [914]:
test_X, test_Y= generateTestData(CURDIGIT=9,TEST_BASELINE_X=TEST_BASELINE_X,TEST_BASELINE_Y=TEST_BASELINE_Y)


(10000, 28, 28)
Original Test Data:
[[   0 1000]
 [   1 1000]
 [   2 1000]
 [   3 1000]
 [   4 1000]
 [   5 1000]
 [   6 1000]
 [   7 1000]
 [   8 1000]
 [   9 1000]]
TestData----After only taking the digit :  9
[[   9 1000]]


In [915]:
extracted_val9,length,LenTestX,patchCount,R_UP9,R2_UP9,R3_UP9  = dataExtraction(curDigitVal=9,test_X=test_X,length=ReadCount,h=4,w=4,IndexArray=INDX_RC)
print(length)
print(len(test_X))
print(patchCount)

print(extracted_val9)
print(temp9)

Extracting total:  126  digits
32/32 [==============================] - 0s 2ms/step
------------- 1 --------------
------------- 2 --------------
------------- 3 --------------
------------- 4 --------------
------------- 5 --------------
------------- 6 --------------
------------- 7 --------------
------------- 8 --------------
------------- 9 --------------
------------- 10 --------------
1000
1000
126
[4, 0, 2, 0, 1, 7, 0, 1, 4, 0, 4, 0, 3, 6, 0, 6, 7, 7, 0, 5, 3, 1, 7, 3, 3, 2, 3, 0, 2, 6, 4, 0, 7, 0, 0, 0, 0, 0, 7, 6, 3, 0, 3, 0, 1, 2, 7, 3, 5, 2, 3, 4, 3, 0, 0, 3, 2, 5, 1, 5, 3, 3, 0, 7, 3, 0, 4, 5, 5, 4, 2, 6, 7, 2, 3, 5, 2, 2, 7, 1, 0, 7, 3, 3, 0, 0, 7, 7, 0, 6, 0, 0, 7, 2, 9, 0, 0, 0, 0, 1, 0, 2, 2, 0, 5, 1, 1, 1, 3, 2, 1, 7, 4, 0, 0, 3, 1, 3, 5, 2, 3, 5, 7, 5, 4, 0]
[4, 5, 2, 5, 1, 1, 2, 5, 2, 0, 4, 0, 3, 6, 0, 0, 7, 7, 0, 7, 6, 1, 4, 3, 2, 6, 1, 5, 7, 6, 4, 0, 7, 1, 0, 0, 0, 7, 5, 1, 3, 6, 5, 7, 1, 2, 7, 6, 5, 5, 1, 6, 3, 0, 0, 6, 3, 5, 7, 5, 2, 7, 5, 7, 6, 1, 1, 5, 6, 4, 7

In [916]:
S=len(extracted_val9)
print(S)

accurate=0
for i in range(len(extracted_val9)):
  if(temp9[i]==extracted_val9[i]):
    accurate+=1

print('send = ',len(temp9),'received = ',len(extracted_val9),'Accurate=', accurate)
print("accuracy ",(accurate*100)/len(temp9),'%')

126
send =  126 received =  126 Accurate= 61
accuracy  48.41269841269841 %


#Retrive All Data


In [917]:
sender_val=temp+temp1+temp2+temp3+temp4+temp5+temp6+temp7+temp8+temp9
#sender_val= temp1

Total_Extracted_Val = extracted_val+ extracted_val1 + extracted_val2 + extracted_val3 + extracted_val4 + extracted_val5 + extracted_val6 + extracted_val7 + extracted_val8 + extracted_val9
#Receiver_val = extracted_val1
Total_R_Up = R_UP+ R_UP1 + R_UP2 + R_UP3 + R_UP4 + R_UP5 + R_UP6 + R_UP7 + R_UP8 + R_UP9
Total_R2_UP = R2_UP+ R2_UP1+ R2_UP2+ R2_UP3+ R2_UP4+ R2_UP5+ R2_UP6+ R2_UP7+ R2_UP8+ R2_UP9
Total_R3_UP = R3_UP+ R3_UP1+ R3_UP2+ R3_UP3+ R3_UP4+ R3_UP5+ R3_UP6+ R3_UP7+ R3_UP8+ R3_UP9
print('Sender sends total: ', len(sender_val),'Receiver receives total: ', len(Total_Extracted_Val))

print(len(Total_Extracted_Val), len(Total_R_Up),len(Total_R2_UP),len(Total_R3_UP))

incorrect=0
for i in range(len(sender_val)):
  if(sender_val[i]!=Total_Extracted_Val[i]):
    incorrect+=1

print('Total incorrect before decoding: ',incorrect)
TOP1_accuracy= (len(sender_val)-incorrect)/len(sender_val)
TOP1_ErrorCount = incorrect
print('Extraction Top-1 Accuracy: ',TOP1_accuracy)

correct=0
for i in range(len(sender_val)):
  if(sender_val[i]==Total_Extracted_Val[i] or sender_val[i]==Total_R_Up[i]):
    correct+=1

print('Extraction Top-2 Accuracy: ',(correct)/len(sender_val))


correct=0
for i in range(len(sender_val)):
  if(sender_val[i]==Total_Extracted_Val[i] or sender_val[i]==Total_R_Up[i] or sender_val[i]==Total_R2_UP[i]):
    correct+=1

print('Extraction Top-3 Accuracy: ',(correct)/len(sender_val))

correct=0
for i in range(len(sender_val)):
  if(sender_val[i]==Total_Extracted_Val[i] or sender_val[i]==Total_R_Up[i] or sender_val[i]==Total_R2_UP[i] or sender_val[i]==Total_R3_UP[i]):
    correct+=1

print('Extraction Top-4 Accuracy: ',(correct)/len(sender_val))


#decodedData = decode(Total_Extracted_Val)


Sender sends total:  1260 Receiver receives total:  1260
1260 1260 1260 1260
Total incorrect before decoding:  460
Extraction Top-1 Accuracy:  0.6349206349206349
Extraction Top-2 Accuracy:  0.7968253968253968
Extraction Top-3 Accuracy:  0.8761904761904762
Extraction Top-4 Accuracy:  0.9230158730158731
